标准库中最常用的一些：

- Box<T>，用于在堆上分配值
- Rc<T>，一个引用计数类型，其数据可以有多个所有者
- Ref<T> 和 RefMut<T>，通过 RefCell<T> 访问，一个在运行时而不是在编译时执行借用规则的类型。

In [2]:
// 使用 Box<T> 在堆上储存数据
fn main() {
    let b = Box::new(5);
    println!("b = {}", b);
}
main()

b = 5


()

In [3]:
async fn do_something() { /* ... */ }


## Box 允许创建递归类型
Rust 需要在编译时知道类型占用多少空间。一种无法在编译时知道大小的类型是 递归类型（recursive type），其值的一部分可以是相同类型的另一个值。这种值的嵌套理论上可以无限的进行下去，所以 Rust 不知道递归类型需要多少空间。不过 box 有一个已知的大小，所以通过在循环类型定义中插入 box，就可以创建递归类型了。

让我们探索一下 cons list，一个函数式编程语言中的常见类型，来展示这个（递归类型）概念。除了递归之外，我们将要定义的 cons list 类型是很直白的，所以这个例子中的概念，在任何遇到更为复杂的涉及到递归类型的场景时都很实用。

In [5]:
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(i32, i32, i32),
}

In [7]:
// List 值最多需要一个 i32 加上 box 指针数据的大小
// box 只提供了间接存储和堆分配；他们并没有任何其他特殊的功能
enum List {    
    Cons(i32, Box<List>),
    Nil,
}

use crate::List::{Cons, Nil};

fn main() {
    let list = Cons(1,
        Box::new(Cons(2,
            Box::new(Cons(3,
                Box::new(Nil))))));
}

main()

()

In [10]:
// 通过解引用运算符追踪指针的值
fn main() {
    let x = 5;
    let y = &x;

    assert_eq!(5, x);
    // 如果尝试编写 assert_eq!(5, y);，则会得到编译错误
    // 不允许比较数字的引用与数字，因为它们是不同的类型。必须使用解引用运算符追踪引用所指向的值。
    assert_eq!(5, *y);
}
main()

()

In [11]:
// 像引用一样使用 Box<T>
fn main() {
    let x = 5;
    let y = Box::new(x);

    assert_eq!(5, x);
    assert_eq!(5, *y);
}
main()

()

In [14]:
use std::ops::Deref;

// 自定义智能指针
struct MyBox<T>(T);

impl<T> MyBox<T> {
    fn new(x: T) -> MyBox<T> {
        MyBox(x)
    }
}

// 通过实现 Deref trait 将某类型像引用一样处理
impl<T> Deref for MyBox<T> {
    type Target = T;

    // deref 方法体中写入了 &self.0，这样 deref 返回了我希望通过 * 运算符访问的值的引用
    // 输入 *y 时，Rust 事实上在底层运行了如下代码：*(y.deref())
    fn deref(&self) -> &T {
        &self.0
    }
}

In [15]:
fn main() {
    let x = 5;
    let y = MyBox::new(x);

    assert_eq!(5, x);
    assert_eq!(5, *y);
}
main()

()

## 函数和方法的隐式解引用强制多态
解引用强制多态（deref coercions）是 Rust 在函数或方法传参上的一种便利。其将实现了 Deref 的类型的引用转换为原始类型通过 Deref 所能够转换的类型的引用。当这种特定类型的引用作为实参传递给和形参类型不同的函数或方法时，解引用强制多态将自动发生。这时会有一系列的 deref 方法被调用，把我们提供的类型转换成了参数所需的类型。

解引用强制多态的加入使得 Rust 程序员编写函数和方法调用时无需增加过多显式使用 & 和 * 的引用和解引用。这个功能也使得我们可以编写更多同时作用于引用或智能指针的代码。

In [16]:
 fn hello(name: &str) {
     println!("Hello, {}!", name);
 }

fn main() {
    let m = MyBox::new(String::from("Rust"));
    // 解引用强制多态使得 Rust 自动的帮我们处理这些转换。
    hello(&m);
}
main()

Hello, Rust!


()

In [18]:
fn main() {
    let m = MyBox::new(String::from("Rust"));
    // 如果 Rust 没有实现解引用强制多态，为了使用 &MyBox<String> 类型的值调用 hello，则不得不编写
    hello(&(*m)[..]);
}
main()

Hello, Rust!


()

## 解引用强制多态如何与可变性交互
类似于如何使用 Deref trait 重载不可变引用的 * 运算符，Rust 提供了 DerefMut trait 用于重载可变引用的 * 运算符。

Rust 在发现类型和 trait 实现满足三种情况时会进行解引用强制多态：

- 当 T: Deref<Target=U> 时从 &T 到 &U。
- 当 T: DerefMut<Target=U> 时从 &mut T 到 &mut U。
- 当 T: Deref<Target=U> 时从 &mut T 到 &U。

头两个情况除了可变性之外是相同的：第一种情况表明如果有一个 &T，而 T 实现了返回 U 类型的 Deref，则可以直接得到 &U。第二种情况表明对于可变引用也有着相同的行为。

第三个情况有些微妙：Rust 也会将可变引用强转为不可变引用。但是反之是 不可能 的：不可变引用永远也不能强转为可变引用。因为根据借用规则，如果有一个可变引用，其必须是这些数据的唯一引用（否则程序将无法编译）。将一个可变引用转换为不可变引用永远也不会打破借用规则。将不可变引用转换为可变引用则需要数据只能有一个不可变引用，而借用规则无法保证这一点。因此，Rust 无法假设将不可变引用转换为可变引用是可能的。